In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import scipy as sp
import collections
import operator
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import networkx.algorithms.community as nx_comm
import altair as alt

In [2]:
#create adjacency matrix
author_list = pd.read_csv('authors_big.csv')
interactions = pd.read_csv('interactions_cleaned.csv')
author_sub_counts = pd.read_csv('author_subreddit_counts_big.csv')

In [120]:
comment_sub_counts = pd.read_csv('comment_subreddit_counts.csv')

In [121]:
political_reddits = ["Conservative", "Republican", "tucker_carlson", "trump", "conservatives", "HillaryForPrison", "ConservativesOnly"]
political_subs = author_sub_counts[author_sub_counts["subreddit"].isin(political_reddits)]

In [122]:
political_subs.head(10)

,author,year,subreddit,post_cnt
1870,-Harboringonalament-,2020,conservatives,1
1896,-Harboringonalament-,2020,Republican,1
10200,1TARDIS2RuleThemAll,2020,Conservative,12
10223,1TARDIS2RuleThemAll,2021,Conservative,4
12295,24identity,2020,trump,1
12455,25QS2,2020,Conservative,3
13659,2horde,2020,Conservative,1
18890,5heepdog,2020,Republican,1
18908,5heepdog,2020,Conservative,1
18921,5heepdog,2021,Conservative,1


In [177]:
political_comms = comment_sub_counts[comment_sub_counts["subreddit"].isin(political_reddits)]

In [178]:
political_comms.head(10)

,author,year,subreddit,comment_cnt
1929,-Automaticity,2020,Conservative,1
5034,-Harboringonalament-,2020,Conservative,1
6142,-MinorWomensWhiplash,2021,Conservative,1
6587,-Nick____,2020,Conservative,4
6652,-Nick____,2020,trump,1
7347,-Philologian,2020,Conservative,1
7466,-Philologian,2021,Conservative,1
7632,-Reddit_Account-,2020,Conservative,1
7922,-Reddit_Account-,2021,Conservative,5
8123,-Russian-Spy-,2020,Conservative,1


In [179]:
political_comms = political_comms[political_comms["comment_cnt"] > 1]

In [180]:
len(political_comms)

3590

In [181]:
len(political_comms.author.unique())

2333

In [126]:
political_comms.sort_values(by=['subreddit'], inplace=True)
political_comms['subreddits'] = political_comms.groupby(['author','year'])['subreddit'].transform(lambda x: ','.join(x))
    

In [127]:
political_comms.head(30)

,author,year,subreddit,comment_cnt,subreddits
6587,-Nick____,2020,Conservative,4,Conservative
2494956,_regionrat,2021,Conservative,30,Conservative
2499009,a1minneapolis,2021,Conservative,2,Conservative
2500574,aSillyPlatypus,2021,Conservative,3,Conservative
2505038,aaronj5467,2020,Conservative,3,Conservative
2505450,aaronrandango2,2020,Conservative,9,Conservative
2505534,aaronrandango2,2021,Conservative,33,Conservative
2510488,acadiel,2021,Conservative,4,Conservative
2514767,adammolens,2020,Conservative,4,Conservative
2487849,__Snafu__,2021,Conservative,5,Conservative


In [128]:
df_two = political_comms[["author", "year", "comment_cnt"]].groupby(["author", "year"]).sum().reset_index()
df = df_two.groupby(["author"]).sum()
df = df[df["year"]==4041]
df = df.reset_index()
#authors = df["author"]

In [129]:
df.head(10)

,author,year,comment_cnt
0,-penismaster,4041,18
1,11711510111411009710,4041,10
2,11Letters1Name,4041,10
3,12h_bottletothrottle,4041,41
4,1320Fastback,4041,101
5,1BigUniverse,4041,21
6,1TARDIS2RuleThemAll,4041,122
7,24identity,4041,9
8,2horde,4041,13
9,47380boebus,4041,66


In [131]:
political_comms_two = political_comms.merge(df["author"], on="author")

In [134]:
political_comms_three = political_comms_two[["author", "year", "subreddits"]].drop_duplicates()

In [141]:
reddit_combs = political_comms_three["subreddits"].value_counts().index.tolist()

In [145]:
year_one_subs = political_comms_three[political_comms_three["year"]==2020]
year_two_subs = political_comms_three[political_comms_three["year"]==2021]

In [146]:
print(len(year_one_subs))

623


In [142]:
print(reddit_combs)

['Conservative', 'Conservative,Republican', 'Conservative,conservatives', 'Republican', 'conservatives', 'tucker_carlson', 'Conservative,trump', 'Conservative,Republican,conservatives', 'trump', 'Conservative,tucker_carlson', 'Conservative,Republican,trump', 'Republican,conservatives', 'Conservative,Republican,tucker_carlson', 'Conservative,ConservativesOnly', 'Conservative,ConservativesOnly,Republican,conservatives', 'Conservative,ConservativesOnly,Republican,conservatives,tucker_carlson', 'HillaryForPrison', 'Conservative,Republican,conservatives,tucker_carlson', 'Conservative,trump,tucker_carlson', 'Conservative,ConservativesOnly,Republican', 'Conservative,conservatives,tucker_carlson', 'Conservative,HillaryForPrison,Republican', 'Conservative,Republican,conservatives,trump', 'Conservative,ConservativesOnly,Republican,conservatives,trump', 'ConservativesOnly', 'Conservative,ConservativesOnly,Republican,conservatives,trump,tucker_carlson', 'Conservative,ConservativesOnly,conservative

In [143]:
print(reddit_combs.index('Conservative'))

0


In [160]:
overlap_matrix = np.zeros((len(reddit_combs), len(reddit_combs)))
for ind in range(len(year_one_subs)):
    overlap_matrix[reddit_combs.index(year_one_subs.iloc[[ind]]["subreddits"].tolist()[0])][reddit_combs.index(year_two_subs.iloc[[ind]]["subreddits"].tolist()[0])] += 1
    
    

In [187]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

overlap_df = []
for x in range(len(reddit_combs)):
    for y in range(len(reddit_combs)):
        d = {
            '2020activity' : reddit_combs[x],  # some formula for obtaining values
            '2021activity' : reddit_combs[y],
            'commSize2020': np.sum(overlap_matrix[x], axis=0),
            'commSize2021': np.sum(overlap_matrix[:, y], axis=0),
            'overlap' : round(float(overlap_matrix[x][y]/np.sum(overlap_matrix[x], axis=0)), 2),
            'overlap_raw' : overlap_matrix[x][y],
        }
        #if np.sum(overlap_matrix[x], axis=0) > 3 and np.sum(overlap_matrix[:, y], axis=0) > 3:
        overlap_df.append(d)

overlap_df = pd.DataFrame(overlap_df)

        
alt.data_transformers.disable_max_rows()
alt.Chart(
    overlap_df,
    title="2010 Daily High Temperature (F) in Seattle, WA"
).mark_rect().encode(
    x='2020activity:O',
    y='2021activity:O',
    color=alt.Color('overlap:Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        alt.Tooltip('2020activity:N', title='Reddits 2020'),
        alt.Tooltip('2021activity:N', title='Reddits 2021'),
        alt.Tooltip('commSize2020:Q', title='Comm Size 2020'),
        alt.Tooltip('commSize2021:Q', title='Comm Size 2021'),
        alt.Tooltip('overlap:Q', title='overlap'),
        alt.Tooltip('overlap_raw:Q', title='overlap raw count')
    ]
).properties(width=550)
        

/var/folders/7v/z00cdktn1vd256t13w0dz0pm0000gn/T/ipykernel_53676/11365913.py:14: RuntimeWarning: invalid value encountered in double_scalars
  'overlap' : round(float(overlap_matrix[x][y]/np.sum(overlap_matrix[x], axis=0)), 2),


alt.Chart(...)

In [183]:
#of these, how many left each subreddit? can you look at particular changes? 
#include people who left

[353.  29.  16.   3.   3.   2.  12.   3.   7.   4.   4.   1.   2.   0.
   0.   0.   0.   0.   4.   0.   0.   0.   2.   0.   0.   1.   0.   1.
   1.   0.   0.   1.   0.   0.   0.   0.   0.   0.]
